In [ ]:
pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 1.8 MB/s eta 0:00:00


In [ ]:
import openai

openai.api_key = ""

In [ ]:
import json

In [ ]:
def get_current_weather(location, unit="섭씨"):
    weather_info = {
        "location": location,
        "temperature": "24",
        "unit": unit,
        "forecast": ["sunny", "windy"],
    }
    return json.dumps(weather_info)

In [ ]:
messages = [{"role": "user", "content": "오늘 서울 날씨 어때??"}]

In [ ]:
functions = [
    {
        "name": "get_current_weather",
        "description": "주어진 지역의 현재 날씨를 알려줍니다.",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "지역, e.g. 서울, 부산, 제주도",
                },
                "unit": {"type": "string", "enum": ["섭씨", "화씨"]},
            },
            "required": ["location"],
        },
    }
]

In [ ]:
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=messages,
    functions=functions,
    function_call="auto",
)


print("==first response==")
print(response)

==first response==
{
  "id": "chatcmpl-7xqoBoeL7SlIiqcBoF1g74PnTeh3D",
  "object": "chat.completion",
  "created": 1694498703,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "get_current_weather",
          "arguments": "{\n  \"location\": \"\uc11c\uc6b8\"\n}"
        }
      },
      "finish_reason": "function_call"
    }
  ],
  "usage": {
    "prompt_tokens": 114,
    "completion_tokens": 18,
    "total_tokens": 132
  }
}


In [ ]:
get_current_weather(location="서울")

'{"location": "\\uc11c\\uc6b8", "temperature": "24", "unit": "\\uc12d\\uc528", "forecast": ["sunny", "windy"]}'

In [ ]:
def get_current_weather(location, unit="fahrenheit"):
    weather_info = {
        "location": location,
        "temperature": "24",
        "unit": unit,
        "forecast": ["sunny", "windy"],
    }
    return json.dumps(weather_info)

messages = [{"role": "user", "content": "지금 서울날씨를 섭씨로 알려줘."}]
functions = [
    {
        "name": "get_current_weather",
        "description": "특정 지역의 날씨를 알려줍니다.",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "지역이름 eg. 서울, 부산, 제주도",
                },
                "unit": {"type": "string", "enum": ["섭씨", "화씨"]},
            },
            "required": ["location"],
        },
    }
]

response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=messages,
    functions=functions,
    function_call="auto",
    )
response_message = response["choices"][0]["message"]

print(response_message)

if response_message.get("function_call"):
    # Note: the JSON response may not always be valid; be sure to handle errors
    available_functions = {
        "get_current_weather": get_current_weather,
    }
    function_name = response_message["function_call"]["name"]
    fuction_to_call = available_functions[function_name]
    function_args = json.loads(response_message["function_call"]["arguments"])
    function_response = fuction_to_call(
        location=function_args.get("location"),
        unit=function_args.get("unit"),
    )

    messages.append(response_message)
    messages.append(
        {
            "role": "function",
            "name": "get_current_weather",
            "content": function_response,
        }
    )

    second_response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=messages,
    )


    json_data = json.dumps(second_response, ensure_ascii=False)

    print(second_response)



{
  "role": "assistant",
  "content": null,
  "function_call": {
    "name": "get_current_weather",
    "arguments": "{\n\"location\": \"\uc11c\uc6b8\",\n\"unit\": \"\uc12d\uc528\"\n}"
  }
}
{
  "id": "chatcmpl-7xqoMuwqFZsjAeiwCmc3CuTi38wJY",
  "object": "chat.completion",
  "created": 1694498714,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "\uc9c0\uae08 \uc11c\uc6b8\uc758 \uae30\uc628\uc740 24\u00b0C\uc774\uace0, \ub9d1\uc740 \ub0a0\uc528\uc774\uba70 \ubc14\ub78c\uc774 \uac15\ud569\ub2c8\ub2e4."
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 107,
    "completion_tokens": 36,
    "total_tokens": 143
  }
}


In [ ]:
second_response.choices[0].message.content

'지금 서울의 기온은 24°C이고, 맑은 날씨이며 바람이 강합니다.'

In [ ]:
json_data

'{"id": "chatcmpl-7xqoMuwqFZsjAeiwCmc3CuTi38wJY", "object": "chat.completion", "created": 1694498714, "model": "gpt-3.5-turbo-0613", "choices": [{"index": 0, "message": {"role": "assistant", "content": "지금 서울의 기온은 24°C이고, 맑은 날씨이며 바람이 강합니다."}, "finish_reason": "stop"}], "usage": {"prompt_tokens": 107, "completion_tokens": 36, "total_tokens": 143}}'